In [35]:
import pandas as pd
import ipywidgets as widgets
pd.set_option('display.max_rows', 20)

In [43]:
board = pd.read_csv('board.csv', header=None)

In [48]:
output = widgets.Output()
with output:
    display(board)
hbox = widgets.HBox([output])
hbox

In [60]:
board[0][0]='1'

In [61]:
output.clear_output()
with output:
    display(board)